In [1]:
!pip install gym==0.25.2 numpy torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.5/734.5 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.7 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.25.2-py3-none-any.whl size=852293 sha256=83d07ca0a6871108baacf7f694074ba6701aaf4df391daaded05c39f72961cae
  Stored in directory: /home/jovyan/.cache/pip/wheels/7e/09/48/68ead2493e0b2c26cb6aeb5a712d379f638b5cc21d3501e2f0
Successfully built gym


In [5]:
import torch

In [8]:
# num employees
num_employee=5

In [22]:
# shift nodes one hot encoded. 6 classes for weekday (one class is dropped to reduce collinearity) and 1 class for night/day shift.
shifts = list()
shifts.append(torch.tensor([0,0,0,0,0,0,0])) # Monday, day
shifts.append(torch.tensor([0,0,0,0,0,0,1])) # Monday, night
shifts.append(torch.tensor([1,0,0,0,0,0,0])) # Tuesday, day
shifts.append(torch.tensor([1,0,0,0,0,0,1])) # Tuesday, night
shifts.append(torch.tensor([0,1,0,0,0,0,0])) # Wednesday, day
shifts.append(torch.tensor([0,1,0,0,0,0,1])) # Wednesday, night
shifts.append(torch.tensor([0,0,1,0,0,0,0])) # Thursday, day
shifts.append(torch.tensor([0,0,1,0,0,0,1])) # Thursday, night
shifts.append(torch.tensor([0,0,0,1,0,0,0])) # Friday, day
shifts.append(torch.tensor([0,0,0,1,0,0,1])) # Friday, night
shifts.append(torch.tensor([0,0,0,0,1,0,0])) # Saturday, day
shifts.append(torch.tensor([0,0,0,0,1,0,1])) # Saturday, night
shifts.append(torch.tensor([0,0,0,0,0,1,0])) # Sunday, day
shifts.append(torch.tensor([0,0,0,0,0,1,1])) # Sunday, night
shifts = torch.vstack(shifts)

In [38]:
# create graph
from torch_geometric.data import HeteroData
data=HeteroData()
data["employee"].x = torch.ones(num_employee)
data["shift"].x = shifts
data['employee', 'has', 'shift'].edge_index = [[1,2],[2,3]]

In [40]:
data

HeteroData(
  employee={ x=[5] },
  shift={ x=[14, 7] },
  (employee, has, shift)={ edge_index=[2] }
)

In [39]:
data['employee', 'has', 'shift'].num_edges

0

In [ ]:
"""
from torch.nn import ReLU, Linear
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops

class GNN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.self_loop = Linear(in_channels, out_channels, bias=False)

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels]

        print(x_i) # ist x_i wirklich eine copy von sich selbst für jeden Edge?

        x_j_2 = self.lin(x_j)
        x_i_2 = self.self_loop(x_i[0])

        x_i_2 * x_j_2.shape[0]

        return x_j_2 + x_i_2

    def update(aggr_out):
        return ReLU(aggr_out)
        
        

gnn_employees = GNN(1,2)
gnn_shifts = GNN(7,2)
"""
        
    

In [25]:
from torch_geometric.nn import RGCNConv

gnn_employees = RGCNConv(in_channels = 1, out_channels=2, num_relations=1)
gnn_shifts = RGCNConv(in_channels = 7, out_channels=2, num_relations=1)

In [ ]:
import itertools

class decoder():
    def __init__(self):
        pass
    def decode(emb_employees, emb_shifts):
        num_employees = emb_employees.shape[0]
        num_shifts = emb_shifts.shape[0]

        scores = list()
        for p in itertools.product(range(num_employees), range(num_shifts)):
            score = torch.dot(emb_employees[p[0]], emb_shifts[p[1]])
            scores.append(((p[0], p[1]),score))

        # returns list of tuples containting the dot product and the indicies of the two nodes
        return scores  